# Generate covariates matrix

In [1]:
suppressMessages({
    library(SummarizedExperiment)
    library(tidyverse)
})

In [2]:
# Function from jaffelab github
merge_rse_metrics <- function(rse) {
    stopifnot(is(rse, 'RangedSummarizedExperiment'))

    rse$overallMapRate = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$overallMapRate, rse$numReads)
    rse$mitoRate = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$mitoRate, rse$numMapped)
    rse$rRNA_rate = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$rRNA_rate, rse$numMapped)
    rse$totalAssignedGene = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$totalAssignedGene, rse$numMapped)

    rse$numMapped = sapply(rse$numMapped, sum)
    rse$numReads = sapply(rse$numReads, sum)
    rse$numUnmapped = sapply(rse$numUnmapped, sum)
    rse$mitoMapped = sapply(rse$mitoMapped, sum)
    rse$totalMapped = sapply(rse$totalMapped, sum)
    return(rse)
}


prep_covs <- function(tissue){
    ancestry = "../../../ancestry_structure/structure.out_ancestry_proportion_raceDemo_compare"
    counts_lt = list("caudate"="../../../counts/_m/caudate_brainseq_phase3_hg38_rseGene_merged_n464.rda", 
                     "dg"="../../../counts/_m/astellas_dg_hg38_rseGene_n263.rda", 
                     'dlpfc'="../../../counts/_m/dlpfc_ribozero_brainseq_phase2_hg38_rseGene_merged_n453.rda", 
                     "hippocampus"="../../../counts/_m/hippo_brainseq_phase2_hg38_rseGene_merged_n447.rda")
    qsv_lt = list("caudate"="../../_m/qSV_caudate.csv", "dg"="../../_m/qSV_dg.csv", 
                  "dlpfc"="../../_m/qSV_dlpfc.csv", "hippocampus"="../../_m/qSV_hippo.csv")
    load(counts_lt[[tissue]])
    rse_df = rse_gene
    keepIndex = which((rse_df$Dx %in% c("Control", "Schizo")) & 
                      rse_df$Race %in% c("AA", "CAUC"))
    rse_df = rse_df[, keepIndex]
    rse_df <- merge_rse_metrics(rse_df)
    colData(rse_df)$RIN = sapply(colData(rse_df)$RIN,"[",1)
    rownames(colData(rse_df)) <- sapply(strsplit(rownames(colData(rse_df)), "_"), "[", 1)
    pheno = colData(rse_df) %>% as.data.frame %>% 
        inner_join(data.table::fread(ancestry), by=c("BrNum"="id", "Race"="group")) %>%
        select(RNum, Eur, Dx, Sex, Age, mitoRate, rRNA_rate, totalAssignedGene, overallMapRate) %>%
        column_to_rownames("RNum")
    mod = model.matrix(~Eur + Dx + Sex + Age + mitoRate + rRNA_rate + 
                       totalAssignedGene + overallMapRate, data = pheno)
    colnames(mod) <- gsub("Dx", "", colnames(mod))
    colnames(mod) <- gsub("SexM", "Male", colnames(mod))
    colnames(mod) <- gsub("Eur", "EA", colnames(mod))
    colnames(mod) <- gsub("\\(Intercept\\)", "Intercept", colnames(mod))
    mod %>% as.data.frame %>% rownames_to_column("RNum") %>% 
        inner_join(data.table::fread(qsv_lt[[tissue]]), by=c("RNum"="V1")) %>% 
        rename_all(list(~str_replace_all(., 'PC', 'qPC'))) %>%
        data.table::fwrite(paste0(tissue, "_covariates.csv"), sep=',')
}

## Main loop

In [3]:
for(tissue in c("caudate", "dg", "dlpfc", "hippocampus")){
    prep_covs(tissue)
}

## Reproducibility Information

In [4]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()

[1] "2021-09-02 16:38:16 EDT"

   user  system elapsed 
 20.484   1.677  22.975 

─ Session info ───────────────────────────────────────────────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 4.0.3 (2020-10-10)
 os       Arch Linux                  
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       America/New_York            
 date     2021-09-02                  

─ Packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────
 package              * version  date       lib source        
 assertthat             0.2.1    2019-03-21 [1] CRAN (R 4.0.2)
 backports              1.2.1    2020-12-09 [1] CRAN (R 4.0.2)
 base64enc              0.1-3    2015-07-28 [1] CRAN (R 4.0.2)
 Biobase              * 2.50.0   2020-10-27 [1] Bioconductor  
 BiocGenerics         * 0.36.1   2021-04-16 [1] Biocondu